In [1]:
import pandas as pd
import yfinance as yf
import funzioni as fx
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

pd.set_option("display.max_columns", None)
elenco_usin = pd.read_excel("DeGiro con formule.xlsx")
elenco_usin = elenco_usin.loc[elenco_usin["Categoria"] != "D", ["USIN", "Ticker"]]
elenco_usin = elenco_usin.sample(frac=1)
model = load_model("LSTM.keras")
usin = elenco_usin.iloc[0]
print(f"\rScarico dati ticker {usin['Ticker']}                                             ", end=" ", flush=True)
ticker = yf.download(usin["USIN"], start='2015-01-01', end='2023-07-31', progress=False)
print(f"\rCalcolo indicatori ticker {usin['Ticker']}                                             ", end=" ", flush=True)
ticker = fx.crea_indicatori(ticker)
features = ticker.drop(columns=["Open", "High", "Low", "Close", "Adj Close", "Max_High_Futuro_20d", "Min_Low_Futuro_20d",
                                "Drawdown_20d", "Perc_Max_High_Futuro_20d", "Perc_Drawdown_20d", "MaxMinRel", "guadagno_max_perc"
                               ])        
target = (ticker["Perc_Max_High_Futuro_20d"] >= 20) & (ticker["Perc_Drawdown_20d"] <=5)
ticker["Target"] = target
X, Y = fx.converti_in_XY(features, target)
Y = np.where(Y, 1, 0)

print(f"\rScalamento dati ticker {usin['Ticker']}                                             ", end=" ", flush=True)
scaler = StandardScaler()
n_samples, n_timesteps, n_features = X.shape
X_reshape = X.reshape((n_samples, n_timesteps * n_features))
X_scaled = scaler.fit_transform(X_reshape)
X_scaled_reshape = X_scaled.reshape((-1, n_timesteps, n_features))

print(f"\rPrevisione {usin['Ticker']}                                             \n", end=" ", flush=True)
pred = model.predict(X_scaled_reshape)

ticker = ticker.iloc[n_timesteps:]
ticker["Pred"] = pred > 0.5
#ticker.to_excel("prova.xlsx")


Previsione SGRY                                                             
58/58 [==============================] - 4s 17ms/step


KeyError: 'target'

In [5]:
cf = pd.DataFrame(confusion_matrix(ticker["Target"], ticker["Pred"]), index=[False, True], columns=["Prev.False", "Prev.True"])
cf

,Prev.False,Prev.True
False,1524,1
True,319,2
